# What do we know about Snowflake?

In [ ]:
CREATE OR REPLACE VIEW DATA.STOCK_PRICES 

AS 

SELECT 


TICKER,
ASSET_CLASS,
PRIMARY_EXCHANGE_CODE,
PRIMARY_EXCHANGE_NAME,
DATE,
"'all-day_high'" ALL_DAY_HIGH,
"'all-day_low'" ALL_DAY_LOW,
"'nasdaq_volume'" NASDAQ_VOLUME,
"'post-market_close'" POST_MARKET_CLOSE,
"'pre-market_open'" PRE_MARKET_OPEN,
YEAR(DATE)::text YEAR,
MONTHNAME(DATE)MONTHNAME, MONTH(DATE) MONTHNO FROM (
SELECT * EXCLUDE VARIABLE_NAME
FROM FINANCE__ECONOMICS.CYBERSYN.STOCK_PRICE_TIMESERIES)


PIVOT (SUM(value) FOR VARIABLE IN (ANY ORDER BY VARIABLE));
SELECT * FROM DATA.STOCK_PRICES WHERE TICKER = 'SNOW'    LIMIT 5

## 01 - Market Place Shareprice Trends
All this is sstraight out the box from the Market Place

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

from snowflake.snowpark.functions import *
from snowflake.snowpark.types import *

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

st.markdown('#### SNOWFLAKE STOCK MARKET PATTERNS')

sdate = st.number_input('Choose Year:',2020,2024,2024)
stock_table = session.table('FINANCE__ECONOMICS.CYBERSYN.STOCK_PRICE_TIMESERIES').filter(year('DATE')==sdate)

def stock_metric(variable, ticker):
    stocks = stock_table.filter((col('TICKER')==ticker) & (col('VARIABLE')==variable))
    stocks = stocks.group_by(col('DATE')).agg(min('VALUE').alias('VALUE')).order_by('DATE')
    return st.line_chart(stocks, y='VALUE',x='DATE', color = '#29B5E8')


ticker = st.selectbox('Select Ticker:',stock_table.select(col('TICKER')).distinct().to_pandas())

col1,col2,col3 = st.columns(3)

with col1:
    st.markdown('#### ALL DAY HIGH')
    stock_metric('all-day_high',ticker)
with col2:
    st.markdown('#### ALL DAY LOW')
    stock_metric('all-day_low',ticker)
with col3:
    st.markdown('#### NASDAQ VOLUME')
    stock_metric('nasdaq_volume',ticker)

col1,col2 = st.columns(2)
with col1:
    st.markdown('#### PRE MARKET OPEN')
    stock_metric('pre-market_open',ticker)
with col2:
    st.markdown('#### POST MARKET CLOSE')
    stock_metric('post-market_close',ticker)



## 02 - Structure the Unstructured
Analyst reports are in a variety of shapes and sizes - via pdf documents - we may however switch to document AI to build a model which we can use to take the data out which we need

In [ ]:
select BUILD_SCOPED_FILE_URL('@DATA.PDFS',RELATIVE_PATH), * from directory(@DATA.PDFS)

### Process Documents from Document AI

In [ ]:
CREATE TABLE if not exists DOCUMENT_AI_PROCESSED AS
SELECT

*,

SNOWFLAKE_BUY_OR_SELL.DOCUMENT_AI.SNOWFLAKE_BUY_SELL!PREDICT(GET_PRESIGNED_URL(@DATA.PDFS,RELATIVE_PATH),2) DOC_META 
FROM DIRECTORY(@DATA.PDFS);

SELECT * EXCLUDE (MD5, ETAG) FROM DOCUMENT_AI_PROCESSED

In [ ]:
CREATE TABLE IF NOT EXISTS REPORTS_STRUCTURED AS 
select RELATIVE_PATH,

DOC_META:__documentMetadata:ocrScore OCR_SCORE,
DOC_META:RATING[0]:value::text RATING,
DOC_META:MARKET_PRICE[0]:value::text MARKET_PRICE,
DOC_META:DATE_REPORT[0]:value::text DATE_REPORT,
DOC_META:NAME_OF_REPORT_PROVIDER[0]:value::text NAME_OF_REPORT_PROVIDER




from DOCUMENT_AI_PROCESSED;

SELECT * FROM REPORTS_STRUCTURED

## Extract ALL text from Analyst reports data
We will use this to make the data searchable.

In [ ]:
CREATE TABLE IF NOT EXISTS DATA.REPORTS AS 

select * exclude layout from (

SELECT *, 

    build_stage_file_url(@DATA.PDFS,RELATIVE_PATH) URL,
    
    SNOWFLAKE.CORTEX.PARSE_DOCUMENT (
                                '@DATA.PDFS',
                                RELATIVE_PATH,
                                {'mode': 'LAYOUT'} )  AS LAYOUT, LAYOUT:content::text CONTENT, LAYOUT:metadata:pageCount PAGE_COUNT
        
                                    FROM DIRECTORY (@DATA.PDFS));

CREATE OR REPLACE VIEW REPORTS_ALL_DATA AS

select A.*, B.CONTENT from REPORTS_STRUCTURED A INNER JOIN 
REPORTS B ON 

A.RELATIVE_PATH = B.RELATIVE_PATH;


SELECT * FROM REPORTS_ALL_DATA

In [ ]:
# Import python packages
import streamlit as st
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark import functions as F
from snowflake.snowpark import types as T



st.title("Equity Research Reports")
session = get_active_session()

side_letters = session.table('SNOWFLAKE_BUY_OR_SELL.DATA.REPORTS_ALL_DATA').select('RELATIVE_PATH').filter(F.col('RELATIVE_PATH').like('ANALYST_REPORTS%'))
file_id = st.selectbox('Select Report:', side_letters)
doc_details = session.table('SNOWFLAKE_BUY_OR_SELL.DATA.REPORTS_ALL_DATA').filter(F.col('RELATIVE_PATH')==file_id).limit(1)
doc_detailsspd = doc_details.to_pandas()


st.markdown('#### Report Details')
col1,col2 = st.columns(2)

with col1:
    st.markdown(f'''__Report Date:__ {doc_detailsspd.DATE_REPORT.iloc[0]}''')
    st.markdown(f'''__Research Firm:__ {doc_detailsspd.NAME_OF_REPORT_PROVIDER.iloc[0]}''')
    
with col2:
    st.markdown(f'''__Close Price Value:__ {doc_detailsspd.MARKET_PRICE.iloc[0]}''')
    st.markdown(f'''__Recommendation:__ {doc_detailsspd.RATING.iloc[0]}''')

# New Section 
import streamlit as st
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark import functions as snow_funcs

import pypdfium2 as pdfium
from datetime import datetime

# Write directly to the app


doc_ai_context = "SNOWFLAKE_BUY_OR_SELL.DATA"
doc_ai_source_table = "REPORTS"
doc_ai_source_verify_table = "REPORTS_ALL_DATA"
doc_ai_doc_stage = "PDFS"

# Dict that has the name of the columns that needs to be verified, it has the column name of the column 
# with value and column with the score
value_dict = {
    "OPERATOR_VALUE": {
        "VAL_COL": "OPERATOR_VALUE",
        "SCORE_COL": "OPERATOR_SCORE"
    }
}

# The minimum score needed to not be verified
threshold_score = 0.5

# HELPER FUNCTIONS
# Function to generate filter to only get the rows that are missing values or have a score below the threshold
def generate_filter(col_dict:dict,  score_val:float): #score_cols:list, score_val:float, val_cols:list):
    
    filter_exp = ''

    # For each column
    for col in col_dict:
        # Create the filter on score threashold or missing value
        if len(filter_exp) > 0:
                filter_exp += ' OR '
        filter_exp += f'(({col_dict[col]["SCORE_COL"]} <= {score_val} ) OR ({col_dict[col]["VAL_COL"]} IS NULL))'

    if len(filter_exp) > 0:
       filter_exp = f'({filter_exp}) AND ' 
    
    # Filter out documents already verified
    filter_exp  += 'verification_date is null'
    return filter_exp

# Generates a column list for counting the number of documents that is missing values or a score less that the threashold
# by each column
def count_missing_select(col_dict:dict, score_val:float):
    select_list = []

    for col in col_dict:
        col_exp = (snow_funcs.sum(
                          snow_funcs.iff(
                                    (
                                        (snow_funcs.col(col_dict[col]["VAL_COL"]).is_null())
                                        | 
                                        (snow_funcs.col(col_dict[col]["SCORE_COL"]) <= score_val)
                                    ), 1,0
                              )
                      ).as_(col)
                )
        select_list.append(col_exp)
        
    return select_list

# Function to display a pdf page
def display_pdf_page():
    pdf = st.session_state['pdf_doc']
    page = pdf[st.session_state['pdf_page']]
            
    bitmap = page.render(
                    scale = 8, 
                    rotation = 0,
            )
    pil_image = bitmap.to_pil()
    st.image(pil_image)

# Function to move to the next PDF page
def next_pdf_page():
    if st.session_state.pdf_page + 1 >= len(st.session_state['pdf_doc']):
        st.session_state.pdf_page = 0
    else:
        st.session_state.pdf_page += 1

# Function to move to the previous PDF page
def previous_pdf_page():
    if st.session_state.pdf_page > 0:
        st.session_state.pdf_page -= 1

# Function to get the name of all documents that need verification
def get_documents(doc_df):
    
    lst_docs = [dbRow[0] for dbRow in doc_df.collect()]
    # Add a default None value
    lst_docs.insert(0, None)
    return lst_docs

# MAIN

# Get the table with all documents with extracted values
df_agreements = session.table(f"{doc_ai_context}.{doc_ai_source_table}")

# Get the documents we already gave verified
df_validated_docs = session.table(f"{doc_ai_context}.{doc_ai_source_verify_table}")

# Join
df_all_docs = df_agreements.join(df_validated_docs,on='RELATIVE_PATH', how='left', lsuffix = '_L', rsuffix = '_R')

# Filter out all document that has missing values of score below the threasholds
validate_filter = generate_filter(value_dict, threshold_score)
df_validate_docs = df_all_docs.filter(validate_filter)
#col1, col2 = st.columns(2)
#col1.metric(label="Total Documents", value=df_agreements.count())
#col2.metric(label="Documents Needing Validation", value=df_validate_docs.count())

# Get the number of documents by value that needs verifying
#select_list = count_missing_select(value_dict, threshold_score)
#df_verify_counts = df_validate_docs.select(select_list)
#verify_cols = df_verify_counts.columns

#st.subheader("Number of documents needing validation by extraction value")
#st.bar_chart(data=df_verify_counts.unpivot("needs_verify", "check_col", verify_cols), x="CHECK_COL", y="NEEDS_VERIFY")

# Verification section
st.divider()
col1, col2 = st.columns(2)
with col1:
    st.markdown('#### RAW PDF STORED IN FILE STORE')
    with st.container():
        # If we have selected a document
        if file_id:        
        # Display the extracted values
            df_doc = df_validate_docs.filter(snow_funcs.col("FILE_NAME") == file_id)
            if 'pdf_page' not in st.session_state:
                st.session_state['pdf_page'] = 0
            if 'pdf_url' not in st.session_state:
                st.session_state['pdf_url'] = file_id    
            if 'pdf_doc' not in st.session_state or st.session_state['pdf_url'] != file_id:
                pdf_stream = session.file.get_stream(f"@{doc_ai_context}.{doc_ai_doc_stage}/{file_id}")
                pdf = pdfium.PdfDocument(pdf_stream)
                st.session_state['pdf_doc'] = pdf
                st.session_state['pdf_url'] = file_id
                st.session_state['pdf_page'] = 0
                
            nav_col1, nav_col2, nav_col3 = st.columns(3)
            with nav_col1:
                if st.button("⏮️ Previous", on_click=previous_pdf_page):
                    pass    
                with nav_col2:
                    st.write(f"page {st.session_state['pdf_page'] +1} of {len(st.session_state['pdf_doc'])} pages")
                with nav_col3:
                    if st.button("Next ⏭️", on_click=next_pdf_page):
                        pass
        
    
    
            display_pdf_page()
    with col2:
        st.markdown('#### EXTRACTED TEXT FROM PDFS')
        with st.container():
            st.markdown(doc_detailsspd.CONTENT.iloc[0])


### Chunk Documents to make the search service easier to find out information
You will see that the document parsing retains all the structure - making it easier to chop up into chunks for searching.  Also, it is fully aware of tables/paragraphs and headings

In [ ]:
CREATE  TABLE IF NOT EXISTS DATA.CHUNKED  AS 

SELECT 
RELATIVE_PATH,
RATING,
DATE_REPORT,
NAME_OF_REPORT_PROVIDER,
SPLIT_PART(RELATIVE_PATH,'/',1)::array DOCUMENT_TYPE,
SPLIT_PART(RELATIVE_PATH,'/',2)::TEXT DOCUMENT, 



VALUE::TEXT TEXT FROM REPORTS_ALL_DATA,LATERAL FLATTEN( SNOWFLAKE.CORTEX.SPLIT_TEXT_RECURSIVE_CHARACTER(CONTENT,'markdown',500,20));

SELECT * FROM DATA.CHUNKED LIMIT 4

#### Add A column to create a financial quarter field

In [ ]:
CREATE TABLE IF NOT EXISTS DATA.ANALYST_REPORTS_CHUNKED AS

SELECT *, REPLACE(SNOWFLAKE.CORTEX.COMPLETE('mistral-large', 
concat('look at the following file name and return only Q2FY25 or Q3FY25.  September is in Q2 and november is in Q3', RELATIVE_PATH,'only return the answer in a format like this - Q3FY25. do not reurn comments' )),' ','')::ARRAY PERIOD, 
SNOWFLAKE.CORTEX.SENTIMENT(TEXT) SENTIMENT


FROM DATA.CHUNKED

In [ ]:
SELECT * FROM DATA.ANALYST_REPORTS_CHUNKED;

### BRING IN THE SOUND DATA

In [ ]:
SELECT * FROM DATA.SUMMARY_TEXT;

In [ ]:
CREATE VIEW IF NOT EXISTS DATA.EARNINGS_CALLS_FORMATTED AS 
SELECT RELATIVE_PATH, 'NA' RATING, 'NA' DATE_REPORT, 'NA' NAME_OF_REPORT_PROVIDER,  ['EARNINGS_CALLS'] DOCUMENT_TYPE,RELATIVE_PATH DOCUMENT, 
TEXT,REPLACE(REPLACE(REPLACE(REPLACE(RELATIVE_PATH,'EARNINGS_',''),'.mp3',''),'_',''),'FY20','FY')::ARRAY PERIOD, SENTIMENT FROM DATA.SUMMARY_TEXT;

SELECT * FROM DATA.EARNINGS_CALLS_FORMATTED LIMIT 3

In [ ]:
CREATE TABLE IF NOT EXISTS DATA.TEXT_AND_SOUND AS 

select * from DATA.EARNINGS_CALLS_FORMATTED

UNION ALL

SELECT * FROM DATA.ANALYST_REPORTS_CHUNKED

### CREATE A SEARCH SERVICE

In [ ]:
CREATE OR REPLACE  CORTEX SEARCH SERVICE STREAMLIT_AND_CORTEX.CHUNKED_REPORTS
  ON TEXT
  ATTRIBUTES DOCUMENT_TYPE,PERIOD
  WAREHOUSE = BUILD_UK_WAREHOUSE
  TARGET_LAG = '1 hour'
  COMMENT = 'SEARCH SERVICE FOR REPORTS - CHUNKED'
  AS SELECT * FROM DATA.TEXT_AND_SOUND;

In [ ]:
USE SCHEMA STREAMLIT_AND_CORTEX;
SHOW CORTEX SEARCH SERVICES